## 测试基于 clickhouse 的策略管理系统接口

---



In [ ]:
import sys
sys.path.insert(0, "A:/ZB/git_repo/waditu/czsc")
import dotenv
dotenv.load_dotenv(dotenv_path="A:/ZB/git_repo/waditu/czsc/.env", override=True)

import os
import czsc
import pandas as pd
from zoneinfo import ZoneInfo


czsc.welcome()

database = "czsc_strategy_test"
tz = ZoneInfo("Asia/Shanghai")

## 初始化策略管理数据库

czsc.cwc.initialize(database=database)

### 策略数据写入

---

1. 写入策略元数据
2. 写入策略持仓权重
3. 写入策略日收益


In [ ]:
czsc.cwc.set_meta(
    strategy="测试策略1号",
    base_freq="日线",
    description="这是一个测试用的策略",
    author="xxx",
    outsample_sdt="20220101",
    weight_type="ts",
    status="实盘",
    memo="",
    overwrite=False,
    database=database,
)

mate = czsc.cwc.get_meta("测试策略1号", database=database)
print(mate)

In [ ]:
dfw = czsc.mock.generate_klines_with_weights()

czsc.cwc.publish_weights(df=dfw, strategy="测试策略1号", database=database, verbose=True)

In [ ]:
wb = czsc.WeightBacktest(dfw, digits=2, fee_rate=0.0)
dfr = pd.melt(wb.daily_return.copy(), id_vars=["date"], var_name="symbol", value_name="returns")
dfr.rename(columns={"date": "dt"}, inplace=True)

print(dfr.tail())

czsc.cwc.publish_returns(df=dfr, strategy="测试策略1号", database=database, verbose=True)

### 读取策略数据
---

In [ ]:
# 读取策略列表

dfm = czsc.cwc.get_all_metas(database=database)
dfm

In [ ]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database)
dfw

In [ ]:
dfw = czsc.cwc.get_strategy_weights(strategy="测试策略1号", database=database, sdt="20230101")
dfw

In [ ]:

dfr = czsc.cwc.get_strategy_returns(strategy="测试策略1号", database=database)
dfr

In [ ]:
dfl  = czsc.cwc.get_latest_weights(database=database)
dfl

In [ ]:
from zoneinfo import ZoneInfo, ZoneInfoNotFoundError


DEFAULT_TZ = ZoneInfo("Asia/Shanghai")

In [ ]:
DEFAULT_TZ